<a href="https://colab.research.google.com/github/lonespear/pubscrabe/blob/main/review_replicate" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install deprecated versions of numpy and spacy for package dependencies
!pip install numpy==1.26.4 > out
!pip install spacy==3.7.0 > out

#Package Bio scrapes pubmed.gov
!pip install biopython > out

# sumy required for abstract summarization
!pip install sumy > out

# for group names and summaries
!pip install transformers > out

!pip install tf-keras --user > out

!pip install bertopic > out


!pip install spacy > out
!python -m spacy download en_core_web_lg > out

!pip install selenium > out
!pip install bs4 > out

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.7.1 requires spacy<3.8.0,>=3.7.2, but you have spacy 3.7.0 which is incompatible.


In [9]:
# to scrape pubmed
from Bio import Entrez
import pandas as pd
from collections import Counter

# for the abstract summarization
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer

# for plotting purposes and K-means
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# used for creating descriptions of they key word cluster
from transformers import pipeline

# For preprocessing and lemmitization of Abstracts
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.util import ngrams
from nltk.tokenize import sent_tokenize
import nltk

# For LDA grouping
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE


# Download the required NLTK resources
nltk.download('punkt')

import spacy
spacy.load('en_core_web_lg')


# for BERTtopic
from bertopic import BERTopic
from transformers import AutoModel, AutoTokenizer
from sentence_transformers import SentenceTransformer

Entrez.email = "jonathan.day@westpoint.edu"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Scraping Functions

In [38]:
def search_pubmed(query, max_results=20000, start_date=None, review_papers_only = False):
    """
    Perform a search on PubMed using the given query and return the results, with a fixed end date and a variable start date.

    :param query: The search query string.
    :param max_results: The maximum number of results to retrieve.
    :param start_date: The earliest publication date to include (YYYY/MM/DD format), default is None.
    :param review_papers_only: Boolean value that picks if the query will only scrape for review papers default is False
    :return: The search results in XML format.
    """
    # Set the fixed end date
    end_date = "2024/09/10" # for reproducibilty, we use a fixed end date, but in pracitce, this should be today's date

    # If a start_date is provided, modify the query to include the date range
    if start_date:
        query += f" AND ({start_date}[Date - Publication] : {end_date}[Date - Publication])"
    else:
        query += f" AND ( 0000/01/01[Date - Publication] : {end_date}[Date - Publication])"

    # Modify query to only pick up review papers
    if review_papers_only:
        query += f" AND (review[pt])"

    # Use the esearch utility to search PubMed
    handle = Entrez.esearch(db="pubmed", term=query, retmax=max_results, usehistory='y')
    # Read the search results
    record = Entrez.read(handle)
    # Close the handle to free up resources
    handle.close()

    # Print the total number of articles found on PubMed for this query
    total_articles = int(record["Count"])
    print(f"Query: {query}")
    print(f"Total articles found for the query: {total_articles}")

    # Extract the list of PubMed IDs (PMIDs) from the search results
    id_list = record["IdList"]

    # Use the efetch utility to fetch details for each PMID
    handle = Entrez.efetch(db="pubmed", id=id_list, retmode="xml")
    # Read the fetched records
    records = Entrez.read(handle)
    # Close the handle to free up resources
    handle.close()

    return records

    # Summarize the abstract of the article
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from nltk.tokenize.punkt import PunktSentenceTokenizer
from nltk.tokenize import word_tokenize

class CustomTokenizer:
    def __init__(self, language="english"):
        self.sentence_tokenizer = PunktSentenceTokenizer()
        self.language = language

    def to_sentences(self, text):
        """Tokenize text into sentences."""
        return self.sentence_tokenizer.tokenize(text)

    def to_words(self, text):
        """Tokenize text into words."""
        return word_tokenize(text, language=self.language)

from sumy.parsers.plaintext import PlaintextParser
from sumy.summarizers.lsa import LsaSummarizer

def summarize_abstract(abstract):
    """
    Summarize the abstract into 1-2 sentences using LSA summarization.

    :param abstract: The abstract of the article.
    :return: A summary of the abstract.
    """
    if not abstract:
        return ""

    # Use the CustomTokenizer
    parser = PlaintextParser.from_string(abstract, CustomTokenizer())
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, 2)  # Summarize to 2 sentences

    return " ".join(str(sentence) for sentence in summary)

def extract_article_info(records):
    """
    Extract core article information from PubMed records, including title, authors, abstract, PubMed ID, URL, references, publication date, and journal.

    :param records: The PubMed records in XML format.
    :return: A DataFrame containing the extracted information.
    """
    data = []

    for article in records["PubmedArticle"]:
        # Extract authors and concatenate their names, if available
        authors_list = []
        if "AuthorList" in article["MedlineCitation"]["Article"]:
            for author in article["MedlineCitation"]["Article"]["AuthorList"]:
                last_name = author.get("LastName", "")
                fore_name = author.get("ForeName", "")
                authors_list.append(last_name + " " + fore_name)
            authors = ", ".join(authors_list)
        else:
            authors = ""

        # Extract the article title
        title = article["MedlineCitation"]["Article"]["ArticleTitle"]

        # Extract the article abstract (or use an empty string if not available)
        abstract = article["MedlineCitation"]["Article"].get("Abstract", {}).get("AbstractText", [""])[0]

        # Extract the PubMed ID
        pubmed_id = article["MedlineCitation"]["PMID"]

        # Construct the PubMed URL
        url = f"https://pubmed.ncbi.nlm.nih.gov/{pubmed_id}/"

        # Extract the journal name
        journal = article["MedlineCitation"]["Article"].get("Journal", {})
        journal_name = journal.get("Title", "")

        # Extract publication date
        pub_date = ""

        if "PubDate" in journal:
            pub_date_obj = journal["PubDate"]
            year = pub_date_obj.get("Year", "")
            month = pub_date_obj.get("Month", "")
            day = pub_date_obj.get("Day", "")
            pub_date = f"{year}-{month}-{day}".strip("-")
        elif "MedlineDate" in journal.get("PubDate", {}):
            pub_date = journal["PubDate"]["MedlineDate"]

        # Summarize the abstract
        summary = summarize_abstract(abstract)

        # Append the extracted data
        data.append({
            "Authors": authors,
            "Title": title,
            "Abstract": abstract,
            "Summary": summary,
            "PublicationDate": pub_date,
            "PubMedURL": url,
            "Journal": journal_name
        })

    # Convert the list of dictionaries to a DataFrame for easier handling
    return pd.DataFrame(data)


In [33]:
query = (
    "((atypia OR benign OR abnormal OR hyperplasia OR proliferative OR histology OR diagnosis OR lesion)[Title/Abstract]) "
    "AND (\"lobular carcinoma in situ\" OR \"lobular neoplasia\" OR \"flat epithelial atypia\" OR \"ductal carcinoma in situ\" "
    "OR \"intraepithelial neoplasia\" OR \"breast lesion\" OR \"atypical hyperplasia\")[Title/Abstract] "
    "AND (cancer OR carcinoma OR malignant OR neoplasia OR tumor OR tumour)[Title/Abstract] "
    "AND (breast OR mammary)[Title/Abstract] "
    "AND (English[Language] AND Humans[MeSH Terms]) "
    "AND (\"2000\"[Date - Publication] : \"2023\"[Date - Publication])"
)

max_results = 100000  # Adjust the number of results to retrieve as needed
n_key_words = 10     # Adjust number of key words you want
n_clusters = 5       # Adjust number of clusters you want for the k-means clustering algorithm

In [34]:
# Perform the search and retrieve the records
records = search_pubmed(query, max_results)

Query: ((atypia OR benign OR abnormal OR hyperplasia OR proliferative OR histology OR diagnosis OR lesion)[Title/Abstract]) AND ("lobular carcinoma in situ" OR "lobular neoplasia" OR "flat epithelial atypia" OR "ductal carcinoma in situ" OR "intraepithelial neoplasia" OR "breast lesion" OR "atypical hyperplasia")[Title/Abstract] AND (cancer OR carcinoma OR malignant OR neoplasia OR tumor OR tumour)[Title/Abstract] AND (breast OR mammary)[Title/Abstract] AND (English[Language] AND Humans[MeSH Terms]) AND ("2000"[Date - Publication] : "2023"[Date - Publication]) AND ( 0000/01/01[Date - Publication] : 2024/09/10[Date - Publication])
Total articles found for the query: 7659


In [39]:
# Parse the retrieved records to extract the desired information
articles_data = pd.DataFrame(extract_article_info(records))

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [25]:
from spacy.lang.en import English

# Create a tokenizer
nlp = English()
tokenizer = nlp.tokenizer

# Tokenize your abstract
abstract = "This is a test abstract. It needs to be tokenized."
tokens = [token.text for token in tokenizer(abstract)]
print(tokens)

['This', 'is', 'a', 'test', 'abstract', '.', 'It', 'needs', 'to', 'be', 'tokenized', '.']
